In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')

In [2]:
turbine_spec = pd.read_csv('csv/turbine.csv')
turbine_spec

,Manufacturer,Model,Rated Power,Rotor Diameter,Hub Height,Cut-in Speed,Cut-out Speed,Rated Wind Speed
0,Enercon,Enercon E103/2.35MW,2350,103,100,2.5,25,12.0
1,Suzlon,Suzlon S97/2.1MW,2100,97,100,4.0,25,12.5
2,Unison,Unison U93/2MW,2000,93,100,3.5,25,11.0


In [3]:
wind_data = pd.read_csv('csv/weibull_fitting.csv')
wind_data

,Unnamed: 0,name,lat,long,avg,c,k,mse
0,0,S05.512_E119.406,-5.512,119.406,6.458895,7.276477,2.141161,0.000175
1,1,S05.512_E119.433,-5.512,119.433,6.412365,7.223399,2.150409,0.000149
2,2,S05.512_E119.460,-5.512,119.460,6.418179,7.228854,2.166980,0.000139
3,3,S05.512_E119.487,-5.512,119.487,6.421796,7.231644,2.164434,0.000135
4,4,S05.512_E119.514,-5.512,119.514,6.412445,7.219531,2.165837,0.000143
...,...,...,...,...,...,...,...,...
139,139,S05.809_E119.595,-5.809,119.595,7.217232,8.114077,2.343465,0.000220
140,140,S05.809_E119.622,-5.809,119.622,7.215866,8.110314,2.362114,0.000238
141,141,S05.809_E119.649,-5.809,119.649,7.206370,8.097183,2.382432,0.000247
142,142,S05.809_E119.676,-5.809,119.676,7.116381,7.994590,2.399925,0.000288


In [4]:
enercon = pd.read_csv('csv/enercon.csv')
suzlon = pd.read_csv('csv/suzlon.csv')
unison = pd.read_csv('csv/unison.csv')

In [5]:
for turbine, loc in zip([enercon, suzlon, unison], [0, 1, 2]):
    turbine['h'] = turbine_spec['Hub Height'].loc[loc]
    turbine['d'] = turbine_spec['Rotor Diameter'].loc[loc]
    turbine['Pwt'] = turbine_spec['Rated Power'].loc[loc]
    turbine['k'] = wind_data['k']
    turbine['c'] = wind_data['c']

In [6]:
for turbine in [enercon, suzlon, unison]:
    turbine.columns = ['lat', 'long', 'nwt', 'D', 'h', 'd', 'Pwt', 'k', 'c']

In [7]:
e_curve = pd.read_csv('csv/enercon_curve.csv')
s_curve = pd.read_csv('csv/suzlon_curve.csv')
u_curve = pd.read_csv('csv/unison_curve.csv')

In [9]:
from scipy.interpolate import interp1d
import scipy.integrate as integrate

In [10]:
# enercon
enercon_power = interp1d(e_curve['v'][1:], e_curve['kW'][1:], fill_value="extrapolate")

def e_function(U):
    return weibull(U) * enercon_power(U) * 8760 * 0.925

e_AEP = []
for i in range(144):
    shape = wind_data['k'].loc[i]
    scale = wind_data['c'].loc[i]
    weibull = lambda U: (shape / scale) * (U / scale)**(shape - 1) * np.exp(-(U / scale)**shape)
    energy = integrate.quad(e_function, 0 , 100, points=e_curve['v'][1:], limit=100)[0]
    e_AEP.append(energy)
    
enercon['AEP (kWh)'] = e_AEP

In [11]:
# suzlon
suzlon_power = interp1d(s_curve['v'][1:], s_curve['kW'][1:], fill_value="extrapolate")

def s_function(U):
    return weibull(U) * suzlon_power(U) * 8760 * 0.925

s_AEP = []
for i in range(144):
    shape = wind_data['k'].loc[i]
    scale = wind_data['c'].loc[i]
    weibull = lambda U: (shape / scale) * (U / scale)**(shape - 1) * np.exp(-(U / scale)**shape)
    energy = integrate.quad(s_function, 0 , 100, points=s_curve['v'][1:], limit=100)[0]
    s_AEP.append(energy)
    
suzlon['AEP (kWh)'] = s_AEP

In [12]:
# unison
unison_power = interp1d(u_curve['v'][1:], u_curve['kW'][1:], fill_value="extrapolate")

def u_function(U):
    return weibull(U) * unison_power(U) * 8760 * 0.925

u_AEP = []
for i in range(144):
    shape = wind_data['k'].loc[i]
    scale = wind_data['c'].loc[i]
    weibull = lambda U: (shape / scale) * (U / scale)**(shape - 1) * np.exp(-(U / scale)**shape)
    energy = integrate.quad(u_function, 0 , 100, points=u_curve['v'][1:], limit=100)[0]
    u_AEP.append(energy)
    
unison['AEP (kWh)'] = u_AEP

In [ ]:
x = np.linspace(0, 30, 100)

fig, ax = plt.subplots(1, figsize=(6,4))
ax.plot(u_curve['v'], u_curve['kW'], label='Unison')
ax.plot(s_curve['v'], s_curve['kW'], label='Suzlon')
ax.plot(e_curve['v'], e_curve['kW'], label='Enercon')
ax.set_xlim(0,27)
ax.set_ylim(0,2500)

ax.text(25.5, 1950, '2000 Unison', color='blue', size=10)
ax.text(25.5, 2050, '2100 Suzlon', color='darkorange', size=10)
ax.text(25.5, 2300, '2350 Enercon', color='green', size=10)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_color('dimgray')
ax.spines['bottom'].set_color('dimgray')
ax.tick_params(colors='dimgray')

ax.set_xlabel('kecepatan angin (m/detik)')
ax.set_ylabel('daya (kW)')
plt.axvline(x=25, color='silver', ls=':')
ax.text(25.5, 100, "cutout speed \npada 25 m/detik", color='grey');

# ax.legend(loc='lower right', frameon=False, fontsize=10, bbox_to_anchor=(0.25, 0.75))

In [13]:
for turbine in [enercon, suzlon, unison]:
    turbine['CF'] = turbine['AEP (kWh)']/(turbine['Pwt']*8760)

In [14]:
import math

def haversine(lat1, long1, lat2, long2):
    # calculate distance between coordinate1(long1, lat1) and coordinate2(long2, lat2)

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(long2 - long1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c / 1000  # output distance in km

In [15]:
# coordinate of two closest grid station

lat_B, long_B = (-5.582, 119.569) # PLN Benteng
lat_T, long_T = (-5.463, 119.474) # Gardu Induk Tallasa

In [16]:
# calculate distance

for turbine in [enercon, suzlon, unison]:
    dt= []
    grid= []
    for i in range(144):
        distance_B = haversine(turbine['lat'].loc[i], turbine['long'].loc[i], lat_B, long_B)
        distance_T = haversine(turbine['lat'].loc[i], turbine['long'].loc[i], lat_T, long_T)
        if distance_B <= distance_T:
            dt.append(distance_B)
            grid.append("PLN Benteng")
        else:
            dt.append(distance_T)
            grid.append("GI Tallasa")
    turbine['dt'] = dt
    turbine['Grid'] = grid
    
for turbine in [enercon, suzlon, unison]:
    turbine['dt(shore)'] = turbine['dt'] * 0.65

In [17]:
# calculate cost

kurs = 16755 * 1000

for turbine in [enercon, suzlon, unison]:
    turbine['Cwt'] = 1.1 * turbine['nwt'] * (2.95 * 10**3 * np.log(turbine['Pwt']/1000) - 375.2)
    turbine['Cf'] = 1.5 * turbine['nwt'] * (320 * turbine['Pwt']/1000 * (1+ 0.02*(turbine['D']*(-1)-8)) * (1+ 0.8*(10**(-6)) * (turbine['h'] * (turbine['d']/2)**2 - 10**5)))
    turbine['Cp'] = 100 / 50.9 * (turbine['Cwt'])
    turbine['Ccs'] = 6.6/100 * turbine['Cp'] * (1+ 0.0023* (turbine['D']*(-1) -10))
    turbine['Cis'] = 7.1/100 * turbine['Cp']
    turbine['Cts'] = 9.1/100 * turbine['Cp'] * (1+ 0.0493*(turbine['dt(shore)']-3)) 
    turbine['Cse'] = 75 * turbine['nwt']
    turbine['Cd'] = 46.8 * turbine['Pwt']/1000 * turbine['nwt']
    turbine['Cost'] = (turbine['Cwt'] + turbine['Cf'] + turbine['Ccs'] + turbine['Cis'] + turbine['Cts'] + turbine['Cse'] + turbine['Cd']) * kurs
    

In [18]:
# economic parameter

t_op = 25       # operating period
t_con = 2       # construction period from 0 year
t_re = 10       # payback period
deg = 1.6/100     # system degradation
OM = 2 /100     # operation and maintenance rate
r_inf = 3/100   # %rate of inflation 
r_d = 4/100     # % discount rate
r_i = 5/100     # % borrowing interest rate
r_c = 80/100    # % borrowing capital rate   
tax = 22/100    # corporate tax rate
price = 1650   

# price of electricity (avg) idr/kWh) 1463(sidrap) 1175(kepmen no 55 k) 1762 (1:1.5 ratio of onshore and offshore)

for turbine in [enercon, suzlon, unison]:
    turbine['Benefit'] = price * turbine['nwt'] * turbine['AEP (kWh)']
    
def invcost(inv, t):
    if t <= t_con:
        return (inv/(t_con) * (1 - r_c))/(1+r_d)**t
    elif t > (t_con + t_re):
        return 0
    else:
        return (r_c * inv / (t_re) * (1+ r_i*(t_con+t_re-t-1)))/(1+r_d)**t
    
def mcost(inv, t):
    if t <= t_con:
        return 0
    else:
        return (inv * OM * (1+r_inf)**t)/(1+r_d)**t
    
def taxcost(aep, t):
    if t <= t_con:
        return 0
    else: 
        return (aep *(1-deg*(t-t_con)) * tax)/(1+r_d)**t
    
def benefit(aep, t):
    if t <= t_con:
        return 0
    else:   
        return ((aep) *(1-deg*(t-t_con)))/(1+r_d)**t

In [19]:
for turbine in [enercon, suzlon, unison]:
    turbine['Cost'] = turbine['Cost'].mask(turbine['nwt']==0, 0)
    turbine['Benefit'] = turbine['Benefit'].mask(turbine['nwt']==0, 0)

In [20]:
bcr_col = np.arange(0,t_op + t_con + 1, 1)
bcr_row = np.arange(0,144,1)
bcr_df = pd.DataFrame(index=bcr_row, columns=bcr_col)
bcr_df['sum'] = 0

e_benefit, s_benefit, u_benefit, e_cost, s_cost, u_cost = [bcr_df.copy() for i in range(6)]

In [21]:
# enercon

for row in bcr_row:
    for time in bcr_col:
        e_benefit.at[row, time] = benefit(enercon['Benefit'].loc[row], time)
        e_cost.at[row, time] = invcost(enercon['Cost'].loc[row], time) + mcost(enercon['Cost'].loc[row], time) + taxcost(enercon['Benefit'].loc[row], time)    
    e_benefit.at[row, 'sum']= sum(e_benefit.loc[row][:-1])
    e_cost.at[row, 'sum']= sum(e_cost.loc[row][:-1])

enercon['BCR'] = e_benefit['sum'] / e_cost['sum']

In [22]:
# suzlon

for row in bcr_row:
    for time in bcr_col:
        s_benefit.at[row, time] = benefit(suzlon['Benefit'].loc[row], time)
        s_cost.at[row, time] = invcost(suzlon['Cost'].loc[row], time) + mcost(suzlon['Cost'].loc[row], time) + taxcost(suzlon['Benefit'].loc[row], time)    
    s_benefit.at[row, 'sum']= sum(s_benefit.loc[row][:-1])
    s_cost.at[row, 'sum']= sum(s_cost.loc[row][:-1])

suzlon['BCR'] = s_benefit['sum'] / s_cost['sum']

In [23]:
# unison

for row in bcr_row:
    for time in bcr_col:
        u_benefit.at[row, time] = benefit(unison['Benefit'].loc[row], time)
        u_cost.at[row, time] = invcost(unison['Cost'].loc[row], time) + mcost(unison['Cost'].loc[row], time) + taxcost(unison['Benefit'].loc[row], time)    
    u_benefit.at[row, 'sum']= sum(u_benefit.loc[row][:-1])
    u_cost.at[row, 'sum']= sum(u_cost.loc[row][:-1])

unison['BCR'] = u_benefit['sum'] / u_cost['sum']

In [24]:
for turbine in [enercon, suzlon, unison]:
    turbine['LCOE (idr/kwh)'] = 1/ (turbine['BCR']/price)

In [25]:
enercon.to_csv("csv/enercon_result.csv")
suzlon.to_csv("csv/suzlon_result.csv")
unison.to_csv("csv/unison_result.csv")

In [26]:
price = 1175
u_benefit.loc[80]['sum']/u_cost.loc[80]['sum']

r_d = 4/100
i = 80

price2 = 1175
unison['Benefit'] = price2 * unison['nwt'] * unison['AEP (kWh)']

breakdown2 = pd.DataFrame(columns=bcr_col, index=['E', 'I', 'M', 'T'])
for time in bcr_col:
    breakdown2.at['E', time] = benefit(unison['Benefit'].loc[i], time)
    breakdown2.at['I', time] = invcost(unison['Cost'].loc[i], time)
    breakdown2.at['M', time] = mcost(unison['Cost'].loc[i], time)
    breakdown2.at['T', time] = taxcost(unison['Benefit'].loc[i], time)
breakdown2.to_csv('csv/breakdown2.csv')

In [33]:
for i in [68, 80, 92, 93]:
    print(unison['Cost'].loc[i]/10**9)


1151.5798609310987
883.0716024837327
971.7717991720286
901.1255646398463


In [ ]:
0.81 